In [1]:
import time
from huggingface_hub import InferenceClient
import json
import requests
import re

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("API_KEY")


In [4]:
import requests
import time

# Hugging Face API URL for the Flan-T5 model
API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-large"

headers = {
    'Authorization': f'Bearer {secret_value_0}',
    'Content-Type': 'application/json'
}

def classify_issue_priority(issue_description, retries=5, wait_time=10):
    """
    Classify the priority of a civic issue using Flan-T5.
    Returns: 'High Priority', 'Medium Priority', or 'Low Priority'
    """

    # Creating a prompt for the Flan-T5 model
    prompt = f"Classify the following civic issue into High, Medium, or Low priority:\n\n\"{issue_description}\""

    # Payload for the API request
    payload = {"inputs": prompt}

    for attempt in range(retries):
        try:
            response = requests.post(API_URL, headers=headers, json=payload)

            if response.status_code == 200:
                result = response.json()
                print("Raw Response:", result)  # Debugging

                # Extract the output text from the model response
                if isinstance(result, list) and 'generated_text' in result[0]:
                    classification = result[0]['generated_text'].strip().lower()
                    
                    # Standardize the output
                    if 'high' in classification:
                        return "High Priority"
                    elif 'medium' in classification:
                        return "Medium Priority"
                    elif 'low' in classification:
                        return "Low Priority"
                    else:
                        return "Unexpected classification: " + classification

                else:
                    print("Unexpected response format:", result)
                    return "Unexpected response format"

            elif response.status_code == 503:
                print(f"Model is loading. Retrying in {wait_time} seconds... (Attempt {attempt + 1})")
                time.sleep(wait_time)
            else:
                print(f"Error: {response.status_code} - {response.text}")
                return None

        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {wait_time} seconds... (Attempt {attempt + 1})")
                time.sleep(wait_time)
            else:
                print("Max retries reached.")
                return None

    print("Model failed to load after several attempts.")
    return None

# 🚀 Test the function with a sample issue
sample_issue = "There is a large pothole on the main road causing traffic jams."
result = classify_issue_priority(sample_issue)
print("Predicted Priority:", result)


Raw Response: [{'generated_text': 'High'}]
Predicted Priority: High Priority
